<a href="https://colab.research.google.com/github/vasukiup/ProcessSequencePrediction/blob/master/SonarDataDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
dataset = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", header=None)
#split into independent and target variables
dataset = dataset.values
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]
#encode class values as integer
encoder = LabelEncoder()
encoder.fit(Y)
encoder_y = encoder.transform(Y)

In [23]:
#create baseline model
def create_baseline():
  model = Sequential()
  model.add(Dense(60, input_shape=(60,), activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_small():
  model = Sequential()
  model.add(Dense(30, input_shape=(60,), activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_larger():
  model = Sequential()
  model.add(Dense(60, input_shape=(60,), activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [15]:
estimator = KerasClassifier(model=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoder_y, cv=kfold)
print ("Baseline: %.2ff%% %.2ff%%" % (results.mean()*100, results.std()*100))

Baseline: 81.17f% 6.94f%


In [13]:
#evaluate the baseline model with standardised dataset
estimators = []
estimators.append(('Standardised', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoder_y, cv=kfold)
print("Standardised: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardised: 85.12% (6.22%)


In [19]:
estimators = []
estimators.append(('Standardised', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_small, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoder_y, cv=kfold)
print("Standardised: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardised: 87.00% (9.10%)


In [24]:
estimators = []
estimators.append(('Standardised', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoder_y, cv=kfold)
print("Standardised: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardised: 84.69% (9.94%)
